In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

from pathlib import Path
from datetime import datetime

# TODO: Create PyNilm python package
import sys
sys.path.append('../') # enabling pynilm import (src content)
sys.path.append('../extras/nilmtk-contrib')

In [2]:
# That's our greatest contribution!!!
from pynilm.data import DataWrapper
from pynilm.experiment import Experiment
from pynilm.models import ConvNet
from pynilm.preprocessing import (
    RQATransformer,
    SequenceToImageTransformer, 
    DeepLearningTransformer
)
from pynilm.settings import (
    SAMPLE_RATE,
    APPLIANCES,
    RP_PARAMS,
    INPUT_SHAPE,
    ESTIMATORS,
    EVALUATION_METRICS
)

In [3]:
# Models
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from pyts.image import (
    RecurrencePlot, 
    GramianAngularField, 
    # MarkovTransitionField
)

In [10]:
# https://dl.acm.org/doi/10.1145/3360322.3360844
# Seq2Point - https://arxiv.org/pdf/1612.09106.pdf
# https://link.springer.com/epdf/10.1007/s00521-020-04916-5?sharing_token=bTZg6CBADDbWx7UVvztexPe4RwlQNchNByi7wbcMAY4YyOCPZ8jI-u3LyC4lDtEOZIQACACm_MVY_633J4jzg0CtjGEkhvPkzOs5Z-2UGgB1P_m1_4nDnPxtIplmNRaDx7TM52V6MVQYVJPSqJEKpxv1n3RqXoEm1ZpW5amjaaA=

for window_size in [
    int((1 * 60) / SAMPLE_RATE), # 1 min
    #int((5 * 60) / SAMPLE_RATE), # 5 mins
    #int((10 * 60) / SAMPLE_RATE), # 10 mins
    #int((15 * 60) / SAMPLE_RATE), # 15 mins
    #int((30 * 60) / SAMPLE_RATE), # 30 mins
    #int((45 * 60) / SAMPLE_RATE), # 45 mins
    #int((1 * 60 * 60) / SAMPLE_RATE), # 1hour
    
    # Empirical tests
    #90,1080,60,
    #720, # 900
    #2040,    
]:
    exp = Experiment(
        model_type='binary',
        models={
            
            #'RQA-MLP': Pipeline([
            #    (
            #        'feature_extractor', RQATransformer(
            #            params=RP_PARAMS,
            #            model_columns=[
            #                "Recurrence rate (RR)", 
            #                "Determinism (DET)"
            #            ]
            #        )
            #    ),
            #    ('scaler', StandardScaler()),
            #    ('estimator', clone(ESTIMATORS['MLP']))
            #]),
            
            #'RP-CNN': Pipeline([
            #    ('rp', SequenceToImageTransformer(
            #        image_transformer=RecurrencePlot(**RP_PARAMS),
            #        image_shape=INPUT_SHAPE,
            #        scaling=True,
            #        flatten_output=False
            #        )
            #    ),
            #    (
            #        'estimator', ConvNet(
            #            input_shape=INPUT_SHAPE,
            #            output_dim=1,
            #            loss_function='binary_crossentropy',
            #            metrics=['accuracy'],
            #            output_activation='sigmoid',
            #            fit_params={
            #                'epochs': 10,
            #                'batch_size': 64,
            #                'verbose': 1
            #            }
            #        )
            #    )
            #]),
            
            #'GASF-CNN': Pipeline([
            #    ('rp', SequenceToImageTransformer(
            #        image_transformer=GramianAngularField(
            #            method='summation',
            #            ),
            #        image_shape=INPUT_SHAPE,
            #        scaling=True,
            #        flatten_output=False
            #        )
            #    ),
            #    (
            #        'estimator', ConvNet(
            #            input_shape=INPUT_SHAPE,
            #            output_dim=1,
            #            loss_function='binary_crossentropy',
            #            metrics=['accuracy'],
            #            output_activation='sigmoid',
            #            fit_params={
            #                'epochs': 10,
            #                'batch_size': 64,
            #                'verbose': 1
            #            }
            #        )
            #    )
            #]),
            
            #'GADF-CNN': Pipeline([
            #    ('rp', SequenceToImageTransformer(
            #        image_transformer=GramianAngularField(
            #            method='difference'
            #            ),
            #        image_shape=INPUT_SHAPE,
            #        scaling=True,
            #        flatten_output=False
            #        )
            #    ),
            #    (
            #        'estimator', ConvNet(
            #            input_shape=INPUT_SHAPE,
            #            output_dim=1,
            #            loss_function='binary_crossentropy',
            #            metrics=['accuracy'],
            #            output_activation='sigmoid',
            #            fit_params={
            #                'epochs': 10,
            #                'batch_size': 64,
            #                'verbose': 1
            #            }
            #        )
            #    )
            #]),
            
            'RP-VGG16-MLP': Pipeline([
                ('rp', SequenceToImageTransformer(
                    image_transformer=RecurrencePlot(**RP_PARAMS),
                    image_shape=INPUT_SHAPE,
                    scaling=True,
                    flatten_output=False
                    )
                ),
                ('dl', DeepLearningTransformer(
                    model='VGG16',
                    weights='imagenet',
                    pooling='avg'
                    )
                ),
                ('estimator', clone(ESTIMATORS['MLP']))
            ]),
            
            #'GASF-VGG16-MLP': Pipeline([
            #    ('rp', SequenceToImageTransformer(
            #        image_transformer=GramianAngularField(
            #            method='summation',
            #            ),
            #        image_shape=INPUT_SHAPE,
            #        scaling=True,
            #        flatten_output=False
            #        )
            #        ),
            #    ('dl', DeepLearningTransformer(
            #
            #        model='VGG16',
            #        weights='imagenet',
            #        pooling='avg'
            #        )
            #    ),
            #    ('estimator', clone(ESTIMATORS['MLP']))
            #]),
            
            #'GADF-VGG16-MLP': Pipeline([
            #    ('rp', SequenceToImageTransformer(
            #        image_transformer=GramianAngularField(
            #            method='difference'
            #            ),
            #        image_shape=INPUT_SHAPE,
            #        scaling=True,
            #        flatten_output=False
            #        )
            #        ),
            #    ('dl', DeepLearningTransformer(
            #        model='VGG16',
            #        weights='imagenet',
            #        pooling='avg'
            #        )
            #    ),
            #    ('estimator', clone(ESTIMATORS['MLP']))
            #]),
            
            },
        
        train_datasource=[
            DataWrapper(
                dataset_path='../data/redd.h5',
                building=3, 
                appliances=APPLIANCES,
                sample_period=SAMPLE_RATE,
                start='2011-04-16 00:00:00',
                end='2011-05-16 23:59:59',
                return_mode='dataframe',
                windows_size=window_size,
                windows_stride=window_size, # No overlapping!,
                get_activations=True,
                activations_type=int,
                debug=False
            )
        ],
        
        test_datasource=[
            DataWrapper(
                dataset_path='../data/redd.h5',
                building=3, 
                appliances=APPLIANCES,
                sample_period=SAMPLE_RATE,
                start='2011-05-17 00:00:00',
                end='2011-05-31 23:59:59',
                return_mode='dataframe',
                windows_size=window_size,
                windows_stride=window_size, # No overlapping!,
                get_activations=True,
                activations_type=int,
                debug=False
            )
        ],
        metrics=EVALUATION_METRICS
    )
    results = exp.run(get_params=False)
    df_results = pd.DataFrame(results)
    df_results.to_excel(f"../extras/multi-nilm/results/window{window_size}-{exp.model_type}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx")

Analyzing `RP-VGG16-MLP` pipeline
Fitting washer dryer model
(44568, 32, 32, 3)
Evaluating accuracy
(19939, 32, 32, 3)
Evaluating f1
(19939, 32, 32, 3)
Evaluating precision
(19939, 32, 32, 3)


In [8]:
df_results.groupby(['model']).agg({
    'accuracy': ['mean'],
    'f1': ['mean'],
    'training_time': ['mean'],
    'prediction_time': ['mean']
}).sort_values(by=[('f1', 'mean')], ascending=False)

,accuracy,f1,training_time,prediction_time
,mean,mean,mean,mean
model,,,,
RP-VGG16-MLP,0.947952,0.594496,31.915023,8.374168


In [9]:
df_results.groupby(['appliance','model']).agg({
    'accuracy': ['mean'],
    'f1': ['mean'],
    'training_time': ['mean'],
    'prediction_time': ['mean']
}).sort_values(by=['appliance', ('f1', 'mean')], ascending=[True, False])

,,accuracy,f1,training_time,prediction_time
,,mean,mean,mean,mean
appliance,model,,,,
dish washer,RP-VGG16-MLP,0.986348,0.496564,31.292872,8.184208
fridge,RP-VGG16-MLP,0.907850,0.907742,31.062320,8.622299
microwave,RP-VGG16-MLP,0.935154,0.483245,31.526280,8.155681
washer dryer,RP-VGG16-MLP,0.962457,0.490435,33.778620,8.534484
